In [ ]:
# class DataSource:        
    
#     def data_preprocess():

#     def data_load_split():

In [1]:
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler

### DataSource

In [2]:
def data_load_split(filename, target=None, ignore=None):
    """
    Args:
        filename: from where to load
        target: list
        ignore: list
        
    Returns:
        X: inputs DataFrame
        y: target DataFrame
    """
    df = pd.read_csv(filename)
    
    target = target
    ignore = ignore
    
    inputs = sorted(set(df.columns) - set(target) - set(ignore))
    
    return df[inputs], df[target]

In [3]:
filename = "./data/titanic_train.csv"
target = ["Survived"]
ignore = ["Name", "Cabin", "Ticket"]

X, y = data_load_split(filename, target, ignore)

In [22]:
def define_problem(y):
    if y.dtypes[0] in ['int64', 'float64'] and y.nunique()[0] == 2:
        problem = "Binary"
    elif y.dtypes[0] in ['object', 'bool']:
        problem = "Classification"
    else:
        problem = "Regression"
    
    return problem

In [23]:
define_problem()

'Binary'

In [6]:
def data_preprocess(X, y, problem="Regression"):
    
    # Data type detection
    numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_ix = X.select_dtypes(include=['object', 'bool']).columns
    
    # Data transform
    num_transform = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    cat_transform = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value="Missing")),
        ('oh_encoder', OneHotEncoder(sparse=False))
    ])
    
    transform_x = ColumnTransformer(transformers=[
        ('num', num_transform, numerical_ix),
        ('cat', cat_transform, categorical_ix)
    ])
    if problem == "Regression":
        transform_y = ColumnTransformer(transformers=[
            ('num', Normalizer(), y.columns)
        ])
    else:
        transform_y = ColumnTransformer(transformers=[
            ('cat', cat_transform, y.columns)
        ])
        
    return transform_x.fit_transform(X), transform_y.fit_transform(y)

In [7]:
trans_X, trans_y = data_preprocess(X, y, problem="Regression")

In [8]:
trans_X, trans_y

(array([[ 0.        , -0.46097065, -0.4852419 , ...,  0.        ,
          0.        ,  0.        ],
        [-0.21422328, -0.46097065, -0.4852419 , ...,  0.        ,
          0.        ,  0.        ],
        [-0.06201858, -0.46097065, -0.4852419 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        , -0.46097065,  0.44524944, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  1.78908189,  6.9586888 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        , -0.46097065, -0.4852419 , ...,  0.        ,
          0.        ,  0.        ]]), array([[0.],
        [0.],
        [1.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [ ]:
# class MLP:
    
#     def __init__():
        
#     def network_frame():
        
#     def build_block():
        
#     def auto_fit():
        
#     def get_param_grid(problem):

In [ ]:
def build_model():
    model
    
    append_dense_layer_block(model, unit)
    
    compile_model(problem, optimizer, loss, metrics)

In [101]:
def build_models(trans_X, trans_y, hidden_layers=3, units=[16]):
    
    models = []
    grid = [np.arange(hidden_layers)+1, units]
    for param_tuple in itertools.product(*grid):
        params = {'hidden_layers': param_tuple[0],
                  'units': param_tuple[1]}
        
        # input layer
        model = keras.Sequential()
        model.add(keras.layers.Dense(16, input_shape=(trans_X.shape[1],)))
        
        # hidden layer block
        for _ in range(params['hidden_layers']):
            model.add(keras.layers.Dense(params['units'], activation='relu'))
        
        # output layer
        model.add(keras.layers.Dense(1, activation='sigmoid'))
        
        models.append(model)
    
    return models

In [129]:
def compile_model(model, problem, optimizer=keras.optimizers.Adadelta, lr=0.02):
    
    # compile network
    if problem == "Binary":
        loss = [keras.losses.binary_crossentropy]
        metrics = ['accuracy']
    elif problem == "Regression":
        loss = [keras.losses.MSE]
        metrics = ['MSE', 'MAE']
    else:
        loss = [keras.losses.categorical_crossentropy]
        metrics = ['accuracy']

    model.compile(optimizer=optimizer(learning_rate=lr),
                  loss=loss,
                  metrics=metrics)
    
    return model

In [ ]:
optimizers = [keras.optimizers.Adadelta, keras.optimizers.SGD,
              keras.optimizers.Adam, keras.optimizers.Adagrad,
              keras.optimizers.Adamax, keras.optimizers.RMSprop]
lrs = [0.001, 0.01, 0.02, 0.1]



for lr in lrs:
    

In [130]:
model_1 = compile_model(models[1], problem="Binary")

In [131]:
model_1.optimizer.lr

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.02>

In [117]:
def create_optimizer():

    optimizer_classes = {'adadelta': keras.optimizers.Adadelta, 'sgd': keras.optimizers.SGD,
                         'adam': keras.optimizers.Adam, 'adagrad': keras.optimizers.Adagrad,
                         'adamax': keras.optimizers.Adamax, 'rmsprop': keras.optimizers.RMSprop}

    opt_class = optimizer_classes.get('adam')
    opt_class(0.2)

In [118]:
opt_class.learning_rate

AttributeError: type object 'Adam' has no attribute 'learning_rate'

In [102]:
models = build_models(trans_X, trans_y, hidden_layers=2, units=[16, 32, 64, 128])

In [103]:
for model in models:
    model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_181 (Dense)            (None, 16)                12496     
_________________________________________________________________
dense_182 (Dense)            (None, 16)                272       
_________________________________________________________________
dense_183 (Dense)            (None, 1)                 17        
Total params: 12,785
Trainable params: 12,785
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_184 (Dense)            (None, 16)                12496     
_________________________________________________________________
dense_185 (Dense)            (None, 32)                544       
__________________________